In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import scipy.spatial as scsp
from knn_plotting import plot
import matplotlib.pyplot as plt
from IPython.display import clear_output

import sys

sys.path.append("../")

from knn_simulation.random_points import uniform_random_points

# Configuration parameters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
eps = np.finfo(np.float32).eps.item()

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
n_query_points = 3
n_data_points = 9
num_actions = (
    4 * n_query_points
)  # we can move up, down, left, right for each query point, note we only carry out one of these for one query point in any step
num_hidden = 128

inputs = layers.Input(shape=(n_data_points * 2,))
common = layers.Dense(num_hidden, activation="relu")(inputs)
action = layers.Dense(num_actions, activation="softmax")(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs=inputs, outputs=[action, critic])

In [ ]:
def calculate_reward(
    data_points, query_points, k_nearest_neighbours, proportion_weighting=2
):

    # set up the tree
    tree = scsp.KDTree(data_points)
    query = tree.query(query_points, k=k_nearest_neighbours)

    # calculate how many points are covered
    covered_points = len(np.unique(query[1]))
    total_points = data_points.shape[0]
    covered = covered_points / total_points

    # if we are covering all points then return 1
    if len(np.unique(query[1])) == len(data_points):
        return 1

    # add a distance to point penalty to stop query points drifting too far away
    max_between_data_points = scsp.distance.pdist(data_points).max()
    mean_query_to_data_point = scsp.distance.cdist(query_points, data_points).mean()

    return (
        proportion_weighting * covered
        - mean_query_to_data_point / max_between_data_points
    )


def action_to_query_point_movement(action_chosen, n_query_points, x_shift, y_shift):
    up = np.array([0, 1])
    left = np.array([-1, 0])
    down = np.array([0, -1])
    right = np.array([1, 0])
    point_to_move = action_chosen // 4
    movement = np.array([up, left, down, right][action_chosen % 4]) * np.array(
        [x_shift, y_shift]
    )
    return np.array(
        [movement if i == point_to_move else [0, 0] for i in range(n_query_points)]
    )

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
huber_loss = keras.losses.Huber()
max_steps_per_episode = 10000
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0
x_bounds = (0, 10)
y_bounds = (0, 10)
grid_size = 20
x_shift = (x_bounds[1] - x_bounds[0]) / grid_size
y_shift = (y_bounds[1] - y_bounds[0]) / grid_size
k_nearest_neighbours = n_data_points / n_query_points
plotting = True
patience = 100

while True:  # Run until solved

    # randomly sample data points
    data_points = uniform_random_points(n_data_points, x_bounds, y_bounds)

    # randomly sample query points at first
    query_points = uniform_random_points(n_query_points, x_bounds, y_bounds)

    episode_reward = 0
    episode_reward_track = []
    with tf.GradientTape() as tape:

        for timestep in range(1, max_steps_per_episode):

            print(
                "Episode: {}, timestep: {}".format(episode_count + 1, timestep),
                end="\r",
            )

            state = tf.convert_to_tensor(data_points.reshape(1, -1))

            # Predict action probabilities and estimated future rewards
            # from environment state
            action_probs, critic_value = model(state)
            critic_value_history.append(critic_value[0, 0])

            # Sample action from action probability distribution
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            action_probs_history.append(tf.math.log(action_probs[0, action]))

            # Apply the sampled action in our environment
            movement = action_to_query_point_movement(
                action, n_query_points, x_shift, y_shift
            )
            query_points = query_points + movement
            reward = calculate_reward(data_points, query_points, k_nearest_neighbours)
            done = reward == 1

            # every 100 steps, we show the result
            if (timestep % 100 == 0) and plotting:
                clear_output(wait=True)
                plot(data_points, query_points, k_nearest_neighbours)
                plt.show()

            rewards_history.append(reward)
            episode_reward += reward
            episode_reward_track += [reward]

            # if we get worse for n steps we stop
            if len(episode_reward_track) > patience and all(
                np.diff(episode_reward_track[-patience:]) < 0
            ):
                done = True

            if done:
                break

        print("\n")

        # Update running reward to check condition for solving
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # Calculate expected value from rewards
        # - At each timestep what was the total reward received after that timestep
        # - Rewards in the past are discounted by multiplying them with gamma
        # - These are the labels for our critic
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # Calculating loss values to update our network
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            # At this point in history, the critic estimated that we would get a
            # total reward = `value` in the future. We took an action with log probability
            # of `log_prob` and ended up recieving a total reward = `ret`.
            # The actor must be updated so that it predicts an action that leads to
            # high rewards (compared to critic's estimate) with high probability.
            diff = ret - value
            actor_losses.append(-log_prob * diff)  # actor loss

            # The critic must be updated so that it predicts a better estimate of
            # the future rewards.
            critic_losses.append(
                huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )

        # Backpropagation
        print("Backpropogating")
        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Clear the loss and reward history
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    # Log details
    episode_count += 1
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))

    if running_reward > 195:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break